## Making Keras Model

think about hyper parameters like Filter size, number of filters, which type of padding to use, which activatioon functions to use etc.

## Libraries

In [1]:
#                                                   LIBRARIES

# Image preprocesssing- processing
import cv2
from PIL import Image
import numpy as np

# Neural Network libraries
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

In [2]:
num_classes = 6
train_animals_reduced = np.load("train_animals_reduced.npy")
train_labels_reduced = np.load("train_labels_reduced.npy")
test_animals_reduced = np.load("test_animals_reduced.npy")
test_labels_reduced = np.load("test_labels_reduced.npy")

In [3]:
# One hot encoding for labels
train_labels_reduced=keras.utils.to_categorical(train_labels_reduced,num_classes)
test_labels_reduced=keras.utils.to_categorical(test_labels_reduced,num_classes)

In [4]:
# import sequential model and all the required layers

#make model
model=Sequential()
# Pairs of Conv2D layer and MaxPool2D Layer with increasing filter sizes ( 16,32 ,64). This helps to make image grow more in depthwise and become more flatten.
# Maxpool: great as they optimize the training time

'''# Pair 1 (16)
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=2))

# Pair 2 (32)
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))

# Pair 3 (64)
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))

# Dropout layers to reduce overfitting
model.add(Dropout(0.2))
model.add(Flatten())


model.add(Dense(500,activation="relu"))
model.add(Dropout(0.2))'''

# capas de calculo -> ir de menor a mayor, suelen ser potencias / multiplos de 2 (las layers)

model.add(Conv2D(filters=64,kernel_size=3,padding="same",activation="relu",input_shape=(50,50,3))) # parte imp = relu
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=128,kernel_size=3,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=256,kernel_size=3,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=4))

model.add(Dropout(0.2))
model.add(Flatten()) # without flatten output shape = (50, 50, 6) -> flatten = (None, 6) which we need to get layer output


# capas de clasificacion

model.add(Dense(200,activation="relu"))

# Final dense layer with num_classes nodes = categories of animals we have in the set
# Softmax activation is used to give scores to these categories which lie between 0 and 1.
model.add(Dense(num_classes,activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 50, 50, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 25, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 25, 25, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 256)       295168    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 3, 3, 256)        0

In [5]:
# compile the model
# We use loss function as categorical_crossentropy and Adam optimizer

# if binary data -> loss = Binary Cross Entropy and activation = sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy'])

In [6]:
# training
# batch potencias de 2, a mayor bacths mayor estabilidad del grandiente
model.fit(train_animals_reduced,train_labels_reduced,batch_size=50 ,epochs=30,verbose=1)

Epoch 1/30
45/45 [==============================] - 8s 148ms/step - loss: 9.1483 - accuracy: 0.2836
Epoch 2/30
45/45 [==============================] - 6s 139ms/step - loss: 1.5197 - accuracy: 0.3813
Epoch 3/30
45/45 [==============================] - 6s 139ms/step - loss: 1.4272 - accuracy: 0.4473
Epoch 4/30
45/45 [==============================] - 6s 139ms/step - loss: 1.3983 - accuracy: 0.4631
Epoch 5/30
45/45 [==============================] - 6s 140ms/step - loss: 1.2704 - accuracy: 0.5224
Epoch 6/30
45/45 [==============================] - 7s 147ms/step - loss: 1.2149 - accuracy: 0.5351
Epoch 7/30
45/45 [==============================] - 6s 143ms/step - loss: 1.1537 - accuracy: 0.5694
Epoch 8/30
45/45 [==============================] - 7s 149ms/step - loss: 1.0360 - accuracy: 0.6165
Epoch 9/30
45/45 [==============================] - 6s 144ms/step - loss: 0.9948 - accuracy: 0.6377
Epoch 10/30
45/45 [==============================] - 6s 144ms/step - loss: 0.9140 - accuracy: 0.6658

In [7]:
# test the model
score = model.evaluate(test_animals_reduced, test_labels_reduced, verbose=1)
print('\n', 'Test accuracy:', score[1])

16/16 [==============================] - 1s 26ms/step - loss: 2.4881 - accuracy: 0.5262

 Test accuracy: 0.5262096524238586


In [8]:
# save the model

'''# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")'''

model.save('model.h5') # Saves the entire model to a single artifact


Saved model to disk


In [9]:
def convert_to_array(img):
    im = cv2.imread(img)
    img = Image.fromarray(im, 'RGB')
    image = img.resize((50, 50))
    return np.array(image)

def get_animal_name(label):
    if label==0:
        return "Bear"
    if label==1:
        return "Chicken"
    if label==2:
        return "Duck"
    if label==3:
        return "Frog"
    if label==4:
        return "Sea Turtle"
    if label==5:
        return "Squirrel"

def predict_animal(file):
    print("Predicting .................................")
    ar=convert_to_array(file)
    ar=ar/255
    label=1
    a=[]
    a.append(ar)
    a=np.array(a)
    score=model.predict(a,verbose=1)
    print(score)
    label_index=np.argmax(score)
    print(label_index)
    acc=np.max(score)
    animal=get_animal_name(label_index)
    print(animal)
    print("The predicted Animal is a "+animal+" with accuracy =    "+str(acc))

In [13]:
# predict image
file = "animals-detection-images-dataset/prediction/turtle.jpg"
predict_animal(file)

Predicting .................................
1/1 [==============================] - 0s 22ms/step
[[0.08369382 0.0450862  0.09178975 0.37404132 0.28347418 0.12191479]]
3
Frog
The predicted Animal is a Frog with accuracy =    0.37404132


In [ ]:
# tensor board -> api para ver en tiempo real la evolicion del modelo (de tensorflow)